In [1]:
# cd /kaggle/working

In [2]:
# !git clone https://github.com/naver/r2d2

In [3]:
# cd r2d2


In [32]:
import os, pdb
import torch
import torch.optim as optim

from tools import common, trainer
from tools.dataloader import *
from nets.patchnet import *
from nets.losses import *
from PIL import Image, ImageOps
from tqdm.notebook import tqdm 


In [33]:
# try:
#     DataPreparation
# except NameError:
#     os.system("cp -r /kaggle/input/naturalimages/ /tmp/dataset")
#     os.system("ln -s /tmp/dataset /kaggle/working/r2d2/data")
#     DataPreparation = True
# else:
#     pass

In [34]:
# !cp -r /kaggle/input/naturalimages/ /tmp/dataset
# !ln -s /tmp/dataset /kaggle/working/r2d2/data

In [35]:
# ls /tmp/dataset

In [36]:
# ls

In [37]:
from  datasets import *

In [38]:
class StillTransform (object):
    """ Takes and return an image, without changing its shape or geometry.
    """
    def _transform(self, img):
        raise NotImplementedError()
        
    def __call__(self, inp):
        img = F.grab_img(inp)

        # transform the image (size should not change)
        try:
            img = self._transform(img)
        except TypeError:
            pass

        return F.update_img_and_labels(inp, img, persp=(1,0,0,0,1,0,0,0))

    
class PixelSpeckleNoise (StillTransform):
    """ Takes an image, and add random white noise.
    """
    def __init__(self, var=.05, seed=None):
        StillTransform.__init__(self)
        assert 0 <= var < 1
        self.var = var
        self.seed = seed

    def __repr__(self):
        return "PixelSpeckleNoise(%g)" % self.var
    
    def normalize(self, img,minimum=0, maximum=1):
        img_max = np.max(img)
        img_min = np.min(img)
        return (img-img_min)/np.abs(img_max-img_min)*(maximum-minimum)+minimum

    def _transform(self, img):
        normalized_img = self.normalize(img)
        upper_band = (12*self.var)**.5
        np.random.seed(self.seed)
        noise = np.random.uniform(-upper_band/2,upper_band/2,size=img.shape)
        noisy_img = normalized_img*(1+noise)
        noisy_img = np.clip(noisy_img,0,1)
        ret_val = self.normalize(noisy_img,maximum=255)
        return  Image.fromarray(np.uint8(ret_val))
    
class PixelNoise (StillTransform):
    """ Takes an image, and add random white noise.
    """
    def __init__(self, ampl=20):
        StillTransform.__init__(self)
        assert 0 <= ampl < 255
        self.ampl = ampl

    def __repr__(self):
        return "PixelNoise(%g)" % self.ampl

    def _transform(self, img):
        img = np.float32(img)
        img += np.random.uniform(0.5-self.ampl/2, 0.5+self.ampl/2, size=img.shape)
        return Image.fromarray(np.uint8(img.clip(0,255)))


In [75]:
default_net = "Quad_L2Net_ConfCFS()"

toy_db_debug = """SyntheticPairDataset(
    ImgFolder('imgs'), 
            'RandomScale(256,1024,can_upscale=True)', 
            'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_web_images = """SyntheticPairDataset(
    web_images, 
        'RandomScale(256,1024,can_upscale=True)',
        'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_aachen_images = """SyntheticPairDataset(
    aachen_db_images, 
        'RandomScale(256,1024,can_upscale=True)', 
        'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_aachen_style_transfer = """TransformedPairs(
    aachen_style_transfer_pairs,
            'RandomScale(256,1024,can_upscale=True), RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_aachen_flow = "aachen_flow_pairs"


db_sar_images = """SyntheticPairDataset(
    sar_db_images, 
        'RandomScale(256,1024,can_upscale=True)', 
        'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""



In [76]:

default_dataloader = """PairLoader(CatPairDataset(`data`),
    scale   = 'RandomScale(256,1024,can_upscale=True)',
    distort = 'ColorJitter(0.2,0.2,0.2,0.1)',
    crop    = 'RandomCrop(192)')"""

default_sampler = """NghSampler2(ngh=7, subq=-8, subd=1, pos_d=3, neg_d=5, border=16,
                            subd_neg=-8,maxpool_pos=True)"""

default_loss = """MultiLoss(
        1, ReliabilityLoss(`sampler`, base=0.5, nq=20),
        1, CosimLoss(N=`N`),
        1, PeakyLoss(N=`N`))"""


In [77]:
data_sources = dict(
#     D = toy_db_debug,
#     W = db_web_images,
#     A = db_aachen_images,
#     F = db_aachen_flow,
#     S = db_aachen_style_transfer,
    X = db_sar_images
    )


In [78]:
class MyTrainer(trainer.Trainer):
    """ This class implements the network training.
        Below is the function I need to overload to explain how to do the backprop.
    """
    def forward_backward(self, inputs):
        output = self.net(imgs=[inputs.pop('img1'),inputs.pop('img2')])
        allvars = dict(inputs, **output)
        loss, details = self.loss_func(**allvars)
        if torch.is_grad_enabled(): loss.backward()
        return loss, details



In [79]:

def load_network(model_fn): 
    checkpoint = torch.load(model_fn)
    print("\n>> Creating net = " + checkpoint['net']) 
    net = eval(checkpoint['net'])
    nb_of_weights = common.model_size(net)
    print(f" ( Model size: {nb_of_weights/1000:.0f}K parameters )")

    # initialization
    weights = checkpoint['state_dict']
    net.load_state_dict({k.replace('module.',''):v for k,v in weights.items()})
    return net.eval()


In [80]:
mkdir trained_models

mkdir: cannot create directory ‘trained_models’: File exists


In [81]:
save_path = "./trained_models"
gpu = 0
train_data = "X"
data_loader = default_dataloader
threads = 8
batch_size = 8
net = default_net
sampler = default_sampler
N = patch_size = 16 
loss = default_loss
learning_rate = 1e-4
weight_decay = 5e-4
epochs = 10
network_path = "./models/faster2d2_WASF_N16.pt"

In [82]:
iscuda = common.torch_set_gpu(gpu)
common.mkdir_for(save_path)


KeyError: 'HOSTNAME'

In [83]:
db = [data_sources[key] for key in train_data]


In [84]:
data_loader.replace('`data`',','.join(db)).replace('\n','')

"PairLoader(CatPairDataset(SyntheticPairDataset(    sar_db_images,         'RandomScale(256,1024,can_upscale=True)',         'RandomTilting(0.5), PixelSpeckleNoise(.5)')),    scale   = 'RandomScale(256,1024,can_upscale=True)',    distort = 'ColorJitter(0.2,0.2,0.2,0.1)',    crop    = 'RandomCrop(192)')"

In [85]:
# Create data loader
db = [data_sources[key] for key in train_data]
db = eval(data_loader.replace('`data`',','.join(db)).replace('\n',''))
print("Training image database =", db)
loader = threaded_loader(db, False, threads, batch_size, shuffle=True)


Training image database = PairLoader
CatPairDataset(Dataset: SyntheticPairDataset   8000 images and pairs   root: data/sar...   Scale: Compose(    RandomScale(0))   Distort: Compose(    RandomTilt(0.5, 'all')    PixelSpeckleNoise(0.5)) )  npairs: 8000
  Distort: ColorJitter(0.2,0.2,0.2,0.1), 
  Crop: RandomCrop((192, 192)), 
  Norm: ToTensor(),  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 



In [86]:
for a in loader:
    print(a)
    break
    

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/javid/.conda/envs/pytorch/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/javid/.conda/envs/pytorch/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/javid/.conda/envs/pytorch/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/run/media/javid/FC201C01201BC216/arco_home/ml/projects/p12_r2d2_local_descriptor/r2d2-master/tools/dataloader.py", line 70, in __getitem__
    img_a, img_b, metadata = self.dataset.get_pair(i, self.what)
  File "/run/media/javid/FC201C01201BC216/arco_home/ml/projects/p12_r2d2_local_descriptor/r2d2-master/datasets/pair_dataset.py", line 

In [70]:
b

'P'

In [31]:
g

In [28]:
next(g)

TypeError: 'NoneType' object is not an iterator

In [ ]:
net = load_network(network_path)

In [ ]:
# # initialization
# pretrained = "./models/faster2d2_WASF_N16.pt"
# checkpoint = torch.load(pretrained, lambda a,b:a)
# net.load_pretrained(checkpoint['state_dict'])


In [34]:
# create losses
loss = loss.replace('`sampler`',sampler).replace('`N`',str(patch_size))
print("\n>> Creating loss = " + loss)
loss = eval(loss.replace('\n',''))



>> Creating loss = MultiLoss(
        1, ReliabilityLoss(NghSampler2(ngh=7, subq=-8, subd=1, pos_d=3, neg_d=5, border=16,
                            subd_neg=-8,maxpool_pos=True), base=0.5, nq=20),
        1, CosimLoss(N=16),
        1, PeakyLoss(N=16))


In [35]:
# create optimizer
optimizer = optim.Adam( [p for p in net.parameters() if p.requires_grad], 
                        lr=learning_rate, weight_decay=weight_decay)

train = MyTrainer(net, loader, loss, optimizer)
if iscuda: train = train.cuda()


AttributeError: 'str' object has no attribute 'parameters'

In [ ]:
# Training loop #
for epoch in range(epochs):
    print(f"\n>> Starting epoch {epoch}...")
    train()

print(f"\n>> Saving model to {save_path}")
torch.save({'net': args.net, 'state_dict': net.state_dict()}, save_path)

